In [1]:
import pandas as pd
import numpy as np
import re

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

talks = pd.read_pickle('../datasets/talksDf.pkl')
print(len(talks))
talks.head()

76633


,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,...,eventType,city,country,eventCode,shortName,year,month,cat,catProbaScore,catName
---M5RE8nJo,El camino a mi trabajo ideal,Eduardo Molina,TEDxAnahuacUniversity,---M5RE8nJo,El camino a mi trabajo ideal | Eduardo Molina ...,4 months ago,65,5,798,"[TEDxTalks, Spanish, Life, Decision making, Li...",...,University,Mexico City,Mexico,31446,AnahuacUniversity,2019,8,2,0.0385,"life, motivation"
--9TX7D0Fu0,Como ser un depresivo exitoso y no morir en el...,Samantha Isabel Aranda Ramírez,TEDxMérida,--9TX7D0Fu0,Como ser un depresivo exitoso y no morir en el...,3 months ago,35,0,819,"[TEDxTalks, Spanish, Health, Depression, Life,...",...,University,Mérida,Mexico,31751,Mérida,2019,9,2,0.0539,"life, motivation"
--KpJ1fatzY,Defusing Stress with Mindful Mojo,Angela & Dennis Buttimer,TEDxGeorgiaStateU,--KpJ1fatzY,Defusing Stress with Mindful Mojo | Angela & D...,2 years ago,60,0,1007,"[TEDxTalks, English, United States, Social Sci...",...,University,Atlanta,United States,24817,GeorgiaStateU,2017,6,5,0.0553,brain
--Nw62FG7-w,The Future of Medical School,"John Tomkowiak, MD",TEDxSpokane,--Nw62FG7-w,"The Future of Medical School | John Tomkowiak,...",2 years ago,20,0,837,"[TEDxTalks, English, Health, Education, Higher...",...,Standard,Spokane,United States,27615,Spokane,2017,12,6,0.0392,technology
--Pkf5htGFM,It's Funny How it Happened,Joey Commisso,TEDxWestVancouverED,--Pkf5htGFM,It's Funny How it Happened | Joey Commisso | T...,3 years ago,11,1,264,"[TEDxTalks, English, Canada, Life, Comedy, Hum...",...,Education,West Vancouver,Canada,27849,WestVancouverED,2016,11,7,0.0338,cities


In [31]:
import pickle
with open('../models/vectorsTalks.pkl', 'rb') as pkl_vec:
    vectorizer = pickle.load(pkl_vec)
toPlot = talks.sample(frac=.01)
print(len(toPlot))
X = vectorizer.transform(toPlot.nouns)

766


In [47]:
from sklearn.metrics.pairwise import linear_kernel
distances = []
for i in range(X.shape[0]):
    distances.append(
        abs(1-linear_kernel(X[i], X).flatten())
    )
distances = np.array(distances)

In [48]:
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=3,metric="precomputed",learning_rate=1000, random_state=123,perplexity=10)
# Y2 = tsne.fit_transform(distances)

In [38]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.manifold import MDS

dim = 3
mds = MDS(n_components=dim, max_iter=3000, eps=1e-9, random_state=0,
                   dissimilarity="precomputed", n_jobs=-1)
Y = mds.fit(distances)

In [150]:
plotData = pd.concat(
    [toPlot, pd.DataFrame(data = Y.embedding_, index = toPlot.index, columns=['x','y','z'])],
    axis=1
)
traces = []

for cname in plotData.catName.unique():
    df = plotData[plotData.catName==cname]
    traces.append(go.Scatter3d(
        name=cname,
        x=df.x,
        y=df.y,
        z=df.z,
        hovertext = "<b style=\"max-width:200px\">"+df.title+"</b><br>"+
                    df.speaker+'<br>'+
                    df.event,
        hoverinfo='text',
        mode='markers',
        marker={
            'size':10
        }
    ))

fig = go.Figure(data=traces)
iplot(fig)